# Advanced Linear Algebra and Optimization

## Imports

In [1]:
import numpy as np
import scipy.linalg as la
import scipy.optimize as opt

## Convergence of a Fixed Markov Chain (Exam)

` In Markov's Chain the row sum must be 1.0 for each of the rows. `

In [2]:
M = np.array(
    [
        [0.7, 0.2, 0.1],
        [0.3, 0.4, 0.3],
        [0.2, 0.3, 0.5],
    ]
)

print(f"Original Matrix: \n {M}")

M_power = M.copy()
tolerance = 1e-9
converged = False
steps = 0

while not converged and steps < 20:
    M_next = M_power @ M_power

    if np.allclose(M_next, M_power, atol=tolerance):
        converged = True
    
    M_power = M_next
    steps += 1

print(f'Converged After {steps} Squaring Steps.')
print(f'Steady State Matrix: \n {M_power}')


Original Matrix: 
 [[0.7 0.2 0.1]
 [0.3 0.4 0.3]
 [0.2 0.3 0.5]]
Converged After 6 Squaring Steps.
Steady State Matrix: 
 [[0.45652174 0.2826087  0.26086957]
 [0.45652174 0.2826087  0.26086957]
 [0.45652174 0.2826087  0.26086957]]


## Convergence of a Random Markov Chain (Exam)

#### Random Matrix Generation and Converting it to a Markov Matrix

In [3]:
random_M = np.random.uniform(0, 1, (3,3))

random_M_sum = np.sum(random_M, axis=1, keepdims=True)
M_rand_sums_1 = random_M / random_M_sum

print(f'Random Markov Matrix: \n {M_rand_sums_1}')
print(f'\nAll Row Checks Sum\n {M_rand_sums_1.sum(axis=1, keepdims=True)}')

Random Markov Matrix: 
 [[0.6748717  0.18150746 0.14362084]
 [0.45307426 0.21724895 0.32967679]
 [0.30109981 0.09828573 0.60061447]]

All Row Checks Sum
 [[1.]
 [1.]
 [1.]]


#### Convergence Test

In [4]:
tolerance = 1e-9
converged = False
steps = 0

while not converged and steps < 20:
    M_next = M_rand_sums_1 @ M_rand_sums_1

    if np.allclose(M_next, M_rand_sums_1, atol=tolerance):
        converged = True
    
    M_rand_sums_1 = M_next
    steps += 1

print(f'Converged After {steps} Squaring Steps.')
print(f'\nSteady State Matrix: \n {M_rand_sums_1}')

Converged After 5 Squaring Steps.

Steady State Matrix: 
 [[0.51980349 0.16065694 0.31953957]
 [0.51980349 0.16065694 0.31953957]
 [0.51980349 0.16065694 0.31953957]]


## Least Squares for Linear Systems (Library)

In [5]:
# Using Both NumPy and SciPy
rows = 4
cols = 2

A = np.array(
    [
        [0, 1],
        [1, 1],
        [2, 1],
        [5, 1],
    ]
)

b = np.array(
    [
        [0],
        [3],
        [3],
        [6],
    ]
)
# x, residuals, rank, s = np.linalg.lstsq(A, b)
x, residuals, rank, s = la.lstsq(A, b)
computed_residual = (b - (A @ x)) ** 2

print(f"Solution: \n {x}")
print(f"\nNumPy Returned Residual Error: {residuals[0]:.5f}")
print(f"Computed Residual Error: {computed_residual.sum():.5f}")

Solution: 
 [[1.07142857]
 [0.85714286]]

NumPy Returned Residual Error: 1.92857
Computed Residual Error: 1.92857


## Least Squares for Overdetermined Linear Systems (Manual)

- $(A^TA)X = A^Tb$
- Residual Error : $ ||b-AX||^2 $

In [6]:
# Manual 
A_T = np.linalg.matrix_transpose(A)
new_A = np.dot(A_T, A)
X = np.linalg.solve(new_A, A_T @ b)

print(f"Solution: \n {x}")
print(f"\nResidual Error: {np.sum((b - (A @ X)) ** 2):.5f}")

Solution: 
 [[1.07142857]
 [0.85714286]]

Residual Error: 1.92857


## Solving Optimization Problems

Maximize $Z = x + 2y - z$ \
`Note: linprog only minimizes. ` <br> <br> To maximize $Z$, we minimize $-Z$. So our objective function becomes $-x - 2y + z$. <br>Constraints: <br> $2x + y + z \le 14$ <br>$4x + 2y + 3z \ge -28 \implies -4x - 2y - 3z \le 28$ (Flip sign for $\le$) <br> $2x + 5y + 5z \le 30$ (Assuming variable order based on context) <br> $x, y, z \ge 0$

In [15]:
c = np.array([-1, -2, 1])  # Objective Function

A_ub = np.array(  # Inequality Constraint Matrix
    [
        [2, 1, 1],
        [-4, -2, -3],
        [2, 5, 5],
    ]
)

b_ub = np.array([14, 28, 30])

x_bound = (0, None)
y_bound = (0, None)
z_bound = (0, None)

res = opt.linprog(c=c, A_ub=A_ub, b_ub=b_ub, bounds=[x_bound, y_bound, z_bound])
# res = opt.linprog(c=c, A_ub=A_ub, b_ub=b_ub, bounds=[x_bound, y_bound, z_bound], method='revised simplex') # deprecated

print(f"Maximized Z Value: {res.fun}")
print(f"Solution for x, y, z: {res.x}")

Maximized Z Value: -13.0
Solution for x, y, z: [5. 4. 0.]
